<a href="https://colab.research.google.com/github/RachitBansal/RedditFlairDetector/blob/master/1_DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection

In this notebook, I have explored the ways to extract data from Reddit submissions. The various methods have been compared and demonstrated.  

In [0]:
!pip install praw

     |████████████████████████████████| 143kB 2.7MB/s 
     |████████████████████████████████| 204kB 8.9MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Using PRAW for collecting the data**

The following was observed:

- A maximum of 250 data points per flair could be collected with this method

- This is extremely low for further processes and thus this method was not used for the final data.

These observations were noted after carrying out the below experiments:

In [0]:
import praw
import pandas as pd
import datetime as dt

In [0]:
import numpy as np
import sklearn.utils as utils

In [0]:
reddit = praw.Reddit(client_id='mZxKduOfxjNThA', \
                     client_secret='s9yiNq1s7URgalN4O8IHqOhCl9w', \
                     user_agent='rClassifier', \
                     username='RachitB2500', \
                     password='RacBan@1')

In [0]:
subreddit = reddit.subreddit('india')

In [0]:
data_dic = {
    'id': [],
    'title': [],
    'disc': [],
    'comment': [],
    'flair': []
}

In [0]:
flairs = ["Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA", "AskIndia"]

In [0]:
for flair in flairs:
  for i, submission in enumerate(subreddit.search('flair:"'+flair+'"', sort='new', syntax='lucene', limit=10000)):
    # submission.flair.select(flairName[1])
    data_dic['id'].append(submission.id)
    data_dic['title'].append(submission.title)
    data_dic['disc'].append(submission.selftext)
    data_dic['flair'].append(flair)
    # try:
    #   top_comment = [comment.body for comment in submission.comments if (hasattr(comment, "body") and comment.distinguished==None)][0]
    # except:
    #   top_comment = ""
    # data_dic['comment'].append(top_comment)
    submission.comments.replace_more(limit=0)
    comments = ''
    for comment in submission.comments.list():
        comments = ' ' + comments + comment.body
    data_dic['comment'].append(comments)
    # try:
    #   for comment in submission.comments[0:1]:
    #     data_dic['comment'].append[comment.body[:150]]
    # except:
    #   data_dic['comment'].append[""]
    if(i%100 == 0):
      print(i)
    if(i%500 == 0):
      print(submission.title)

0
Siddis: In It For The Long Run | Unique Stories from India
100
200
0
Pakistan’s former legislator seeks asylum in India
100
200
0
Random Daily Discussion Thread - April 10, 2020 at 09:15AM
100
200
0
Something refreshing to look at during these lockdown times
100
200
0
Bottle stand that keeps you hydrated
100
200
0
How Modi Has Trampled the Founding Idea of India - The Atlantic
100
200
0
ADB Assures $ 2.2 Billion Support To India During Coronavirus Pandemic
100
200
0
99.8% workforce in IT sector incapable of remote working: Study
100
200
0
India turns to Taiwan for cricket
100
200
0
What to do if some shopkeepers in my area are selling products at prices above MRP in this pandemic?
100
200
0
Hi, I’m Dr. Roshan Radhakrishnan, a doctor, and an avid writer. I am here to answer your queries about Coronavirus and then some more. AMA.
100
200
0
What is the state of online deliveries - I need to order a few items?
100
200


In [0]:
# data_dic['id'] = data_dic['id'][1:]
rows = min(len(data_dic[key]) for key in list(data_dic.keys()))
for key in list(data_dic.keys()):
  # print(len(data_dic[key]))
  if(len(data_dic[key]) > rows):
    data_dic[key] = data_dic[key][len(data_dic[key]) - rows:]

In [0]:
data = pd.DataFrame(data_dic)
df = data.sample(frac=1).reset_index(drop=True)

In [0]:
df.to_csv('drive/My Drive/redditIndia_1.csv')

In [0]:
df.head(20)

,id,title,disc,comment,flair
0,cziid8,I have a flight to catch at 10.30 PM. But can'...,I booked a flight to Muscat through MakeMyTrip...,MMT & Airlines will charge you for that un...,[R]eddiquette
1,fxs1vy,Politics in the time of corona: WB CM question...,,"Oh FFS. \n\nYellow, Orange...",Politics
2,dar2o6,Food insecurity affects children’s learning,,Already knew that. Just watching the kid pick...,Food
3,esokhg,Is there really any way to differentiate cage ...,In my city they sell 6 cage free eggs for 100 ...,I think that cage free eggs have a dens...,Food
4,d0c9gt,Noida family gets threat for letting girl ride...,,Here’s a...,[R]eddiquette
5,ffo1yk,"YES Bank's Rs 10,800 crore bonds could soon tu...",,,Business/Finance
6,ftihsj,What technologies are needed to become a full ...,"Hello, I am interested in becoming a full stac...",Currently learning MERN and/or MEAN is the ea...,Non-Political
7,fvblal,Saharanpur Police says no truth in Tablighi Ja...,,Here’s anot...,Politics
8,fss2u3,"Despite 3-mth moratorium announcement, borrowe...",,Very messy implementation. The RBI should ha...,Policy/Economy
9,fthfk5,reminder that today might be the last day to s...,Tomorrow Hotstar will be rebranded to disney+ ...,Me gareeb using a free streami...,Non-Political


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2925 non-null   object
 1   title    2925 non-null   object
 2   disc     2925 non-null   object
 3   comment  2925 non-null   object
 4   flair    2925 non-null   object
dtypes: object(5)
memory usage: 114.4+ KB


In [0]:
for flair in flairs:
  print(flair, "\t", np.sum(df.loc[:, 'flair'].values == flair))

275
232
240
241
244
248
247
250
222
243
235
248


In [0]:
for key in list(data_dic.keys()):
  print(key, "\t", np.sum(df.loc[:, key].values != ""))

id 	 2925
title 	 2925
disc 	 1258
comment 	 2460
flair 	 2925


**Using pushshift for Data Collection**

The following was observed:

- Though this API allows a maximum of 1000 results per call, repeated calls after changing the timestamp allows us to gather a huge amount of data.

- A bit around .7M data points have been scraped and this, after preprocessing, would act as the final data. 

In [0]:
import requests

In [0]:
net_json = {'data': []}
glob_ids = []

In [0]:
def get_pushshift_data(size, sort_type, sort, fields, subreddit, query = None, after = None, before = None):
  if(before):
    if(after):
      url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&fields={fields}&size={size}&sort={sort}&sort_type={sort_type}&after={after}&before={before}"
    else:
      url = f"https://api.pushshift.io/reddit/search/submission/?query={query}&subreddit={subreddit}&fields={fields}&size={size}&sort={sort}&sort_type={sort_type}&before={before}"

  elif(query):
    url = f"https://api.pushshift.io/reddit/search/submission/?query={query}&subreddit={subreddit}&fields={fields}&size={size}&sort={sort}&sort_type={sort_type}"

  else:
    url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&fields={fields}&size={size}&sort={sort}&sort_type={sort_type}"

  request = requests.get(url)
  json_response = request.json()
  # print(json_response)/
  for i in range(len(json_response['data'])):
    sub_id = json_response['data'][i]['id']
    if(sub_id not in glob_ids):
      glob_ids.append(sub_id)
      net_json['data'].append(json_response['data'][i])
    
  return int(net_json['data'][-1]['created_utc'])

In [0]:
data_type = 'submission'
subreddit = 'India'
flairs = ["Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA", "AskIndia"]
fields = 'id,title,selftext,url,link_flair_text,num_comments,created_utc'
size = 1000
last = 1586721116       # time when this was written 

for i in range(1, 50):
  if(i%10 == 0):
    print(i)
  last = get_pushshift_data(size=1000, sort_type='created_utc', sort='desc', fields = fields, subreddit = subreddit, before=last)

10
20
30
40


In [0]:
print(len(net_json['data']))
net_json['data'][-1]

49000


{'created_utc': 1577977801,
 'id': 'eizd3j',
 'link_flair_text': 'AskIndia',
 'num_comments': 0,
 'selftext': '[removed]',
 'title': 'Vape related',
 'url': 'https://www.reddit.com/r/india/comments/eizd3j/vape_related/'}

In [0]:
for i in range(51, 150):
  if(i%10 == 0):
    print(i)
  last = get_pushshift_data(size=1000, sort_type='created_utc', sort='desc', fields = fields, subreddit = subreddit, before=last)

60
70
80
90
100
110
120
130
140


In [0]:
print(len(net_json['data']))
net_json['data'][-1]

148000


{'created_utc': 1560296121,
 'id': 'bzjufa',
 'link_flair_text': 'Politics',
 'num_comments': 4,
 'selftext': '',
 'title': 'Another Journalist Arrested for ‘Defamatory Content’ Against Yogi',
 'url': 'https://www.thequint.com/news/india/another-journalist-held-for-defamatory-content-against-yogi-adityanath'}

In [0]:
for i in range(151, 300):
  if(i%10 == 0):
    print(i)
  last = get_pushshift_data(size=1000, sort_type='created_utc', sort='desc', fields = fields, subreddit = subreddit, before=last)

160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [0]:
print(len(net_json['data']))
net_json['data'][-1]

297000


{'created_utc': 1536236060,
 'id': '9dhwws',
 'num_comments': 0,
 'selftext': '',
 'title': 'Want to drink coffee alongside snakes? Head to this Cambodia-cafe!',
 'url': 'https://www.newsbytesapp.com/timeline/World/31298/139425/this-cambodia-cafe-offers-coffee-and-reptiles'}

In [0]:
last = 1522540800
for i in range(200):
  if(i%10 == 0):
    print(i)
  last = get_pushshift_data(size=1000, sort_type='created_utc', sort='desc', fields = fields, subreddit = subreddit, before=last)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [0]:
print(len(net_json['data']))
net_json['data'][-1]

697000


{'created_utc': 1491756898,
 'id': '64e1gj',
 'link_flair_text': 'Not Original Title',
 'num_comments': 3,
 'selftext': '[deleted]',
 'title': 'TIL Contraceptives have been banned as OTC drugs in Tamil Nadu for over a decade. So much progression..',
 'url': 'http://m.timesofindia.com/city/chennai/Online-petition-seeks-revocation-of-ban-on-over-the-counter-sale-of-emergency-contraceptive-pills-in-TN/articleshow/54703048.cms'}

In [0]:
df = pd.DataFrame(net_json['data'])
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,created_utc,id,link_flair_text,num_comments,selftext,title,url
0,1554185977,b8fe48,NaN,2,,Opinion | A theory about the sudden rise of Ni...,https://www.livemint.com/opinion/columns/opini...
1,1515991106,7qhc9y,NaN,0,[deleted],PU Colleges in Mangalore.,https://www.reddit.com/r/india/comments/7qhc9y...
2,1558945602,btjiv5,Politics,3,,Remembering Jawaharlal Nehru on his death anni...,https://i.redd.it/txph3eq1sp031.jpg
3,1501064512,6pn4gj,Non-Political,0,,[FRESH VIDEO] Armaan Malik - 'Barfani' (Babumo...,https://www.youtube.com/watch?v=LzETSqumgCY
4,1517726535,7v59mr,NaN,0,,"7,000 super-rich Indians shifted overseas in 2...",http://zeenews.india.com/india/7000-super-rich...
5,1574087019,dy3t9k,Non-Political,2,Today I had an appointment for my kid's passpo...,Whims and fancies of officers at PSK [NP],https://www.reddit.com/r/india/comments/dy3t9k...
6,1410478029,2g5gqn,Non-Political,1,,Rann Utsav - A festival in the deserts of Kutc...,http://www.rannutsav.net/
7,1483609441,5m5j3f,Science/Technology,3,,CCMB signs MoUs with four startups,http://economictimes.indiatimes.com/small-biz/...
8,1479750176,5e5caf,Politics,0,[deleted],PM Modi didn't do his homework,http://www.forbes.com/sites/panosmourdoukoutas...
9,1492613889,66aplw,Politics,2,,GST rules: One leg still stuck in old taxes,http://www.forbesindia.com/article/special/gst...


In [0]:
flairs_in_df = list( dict.fromkeys(list(df.loc[:, 'link_flair_text'].values)))
print(len(flairs_in_df))
for flair in flairs_in_df:
  count = np.sum(df.loc[:, 'link_flair_text'].values == flair)
  if(count>500):
    print(flair, "\t", count)

707
AskIndia 	 45811
Politics 	 129061
Food 	 3395
Coronavirus 	 7004
Photography 	 4331
Business/Finance 	 17436
Policy/Economy 	 18642
Non-Political 	 151076
Sports 	 6217
Low-effort self-post. 	 1231
Science/Technology 	 14010
Scheduled 	 1754
Meta. 	 529
All CAPS. 	 1700
Demonetization 	 6457
CAA-NRC 	 1070
Science &amp; Technology 	 1810
[R]eddiquette 	 10424
Not in English. 	 4666
Entertainment 	 4421
Not about India. 	 2615
Repost. 	 2307
Not Original Title. 	 2117
Stickied Topic. 	 799
Policy 	 2306
Policy &amp; Economy 	 2012
Business &amp; Finance 	 892
Repost; 	 1284
Unverified Content. 	 930


In [0]:
 df.to_csv('./drive/My Drive/rMIDAS_final.csv', index = False)